# 利用rating2inter.ipynb中U/I的index对features进行一一对应(meta-text)
- Reindex item feature ID with IDs generated in 0rating2inter.ipynb

In [ ]:
import os
import pandas as pd

In [ ]:
os.chdir('../data/Baby')
os.getcwd()

In [ ]:
# load item mapping
i_id_mapping = 'i_id_mapping.csv'
df = pd.read_csv(i_id_mapping, sep='\t')
print(f'shape: {df.shape}')
df[:4]

In [ ]:

import gzip, json
meta_file = 'meta_Baby.json.gz'

print('0 Extracting U-I interactions.')

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF(meta_file)

print(f'Total records: {meta_df.shape}')
meta_df[:3]

In [ ]:
# remapping
map_dict = dict(zip(df['asin'], df['itemID']))

meta_df['itemID'] = meta_df['asin'].map(map_dict)
meta_df.dropna(subset=['itemID'], inplace=True)
meta_df['itemID'] = meta_df['itemID'].astype('int64')
#meta_df['description'] = meta_df['description'].fillna(" ")
meta_df.sort_values(by=['itemID'], inplace=True)

print(f'shape: {meta_df.shape}')
meta_df[:2]

In [ ]:
ori_cols = meta_df.columns.tolist()

ret_cols = [ori_cols[-1]] + ori_cols[:-1]
print(f'new column names: {ret_cols}')

In [ ]:
meta_df[:3]

In [ ]:
ret_df = meta_df[ret_cols]
# dump
ret_df.to_csv(os.path.join('./', 'meta-baby.csv'), index=False)
print('done!')

## Reload

In [ ]:
indexed_df = pd.read_csv('meta-baby.csv')
print(f'shape: {indexed_df.shape}')
indexed_df[:4]

In [ ]:
## Reload

i_uni = indexed_df['itemID'].unique()

print(f'# of unique items: {len(i_uni)}')

print('min/max of unique learners: {0}/{1}'.format(min(i_uni), max(i_uni)))